## 8.2 Inferencing Faster R-CNN
- Inferencing faster R-CNN OpenCV DNN
- Inferencing faster R-CNN ONNX Runtime


> ⚠️⚠️⚠️ <font color="orange">Run this notebook in your VS Code</font> ⚠️⚠️⚠️

- Downgrade OpenCV to version 4.7.x
    - This downgrade is required due to Issue related to ONNX YoloV8 in OpenCV DNN for version >= 4.8.x
        - https://github.com/ultralytics/ultralytics/issues/1836
    - Open Anaconda Prompt 
    - Activate environment
    ```
    conda activate BelajarOpenCV
    ```
    - Downgrade to OpenCV 4.7.x
    ```
    pip install --force-reinstall opencv-python==4.7.0.72 --user
    ```


- Install necesarry package, on conda prompt, execute
    ```
    pip install onnx
    pip install onnxruntime
    ```

### 8.2.1 Inferencing faster R-CNN OpenCV DNN
> ⚠️⚠️⚠️ <font color="orange">OpenCV DNN with version 4.7 - 4.10 still not supported implementation of Faster R-CNN model in ONNX format</font> ⚠️⚠️⚠️<br>
>
> Error OpenCV 4.7 : <font color="#e65643"><i>Input node with name /rpn/anchor_generator/Gather_output_0 not found in function 'cv::dnn::Subgraph::getInputNodeId</i></font><br>
> Error OpenCV 4.9 & 4.10 : <font color="#e65643"><i>concat_layer.cpp:104: error: (-215:Assertion failed) curShape.size() == outputs[0].size() in function 'cv::dnn::ConcatLayerImpl::getMemoryShapes</i></font><br><br>
>
> <i>We are keeping the code commented below until the update is available</i>.

In [ ]:
# import numpy as np
# import cv2

# cv2.__version__

In [ ]:
# def load_model(onnx_model_path):
#     # Load the ONNX model
#     net = cv2.dnn.readNetFromONNX(onnx_model_path)
#     return net

# def preprocess_image(image_path):
#     # Load the image and resize to 224x224
#     image = cv2.imread(image_path)
#     image_resized = cv2.resize(image, (224, 224))
#     blob = cv2.dnn.blobFromImage(image_resized, 1/255.0, (224, 224), swapRB=True, crop=False)
#     return image, image_resized, blob

# def run_inference(net, blob):
#     # Set the input to the network
#     net.setInput(blob)
    
#     # Forward pass through the network
#     outputs = net.forward()  # Get all the output layers
#     return outputs

# def post_process(outputs, image_shape, threshold=0.5):
#     boxes, labels, scores = [], [], []
    
#     # Assuming output format: boxes, labels, scores
#     for output in outputs:
#         # Iterate through each detected object
#         for detection in output:
#             confidence = detection[2]
#             if confidence > threshold:
#                 # Extract box coordinates (x_min, y_min, x_max, y_max)
#                 box = detection[3:7] * np.array([image_shape[1], image_shape[0], image_shape[1], image_shape[0]])
#                 boxes.append(box.astype("int"))
#                 labels.append(int(detection[1]))
#                 scores.append(float(confidence))
    
#     return boxes, labels, scores

# def draw_detections(image, boxes, labels, scores, class_names=None):
#     for box, label, score in zip(boxes, labels, scores):
#         # Draw the bounding box
#         cv2.rectangle(image, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
#         label_text = f"{class_names[label]}: {score:.2f}" if class_names else f"Label {label}: {score:.2f}"
        
#         # Put the label text
#         cv2.putText(image, label_text, (box[0], box[1] - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    
#     return image

# # Main function to perform inference
# def main(image_path, onnx_model_path):
#     # Load the model
#     net = load_model(onnx_model_path)
    
#     # Preprocess the image
#     original_image, resized_image, blob = preprocess_image(image_path)
    
#     # Run inference
#     outputs = run_inference(net, blob)
    
#     # Post-process the outputs
#     boxes, labels, scores = post_process(outputs, original_image.shape)
    
#     # Define class names if available
#     class_names = ["background", "scissors"]  # Modify according to your dataset

#     # Draw the detections on the original image
#     result_image = draw_detections(original_image.copy(), boxes, labels, scores, class_names)
    
#     # Display the result
#     cv2.imshow("Detections", result_image)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

# # Example usage
# image_path = "image2.jpg"
# main(image_path, MODEL_NAME)


<br><br><br><br>
### 8.2.2 Inferencing Faster R-CNN ONNX Runtime

In [ ]:
import onnx
import onnxruntime as ort

import numpy as np
import cv2

- Load sample image

In [ ]:
# Load your image
image_path = "image2.jpg"
original_image = cv2.imread(image_path)
image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
image = cv2.resize(image, (224, 224))

# normalize image
image_norm = (image / 255.0).astype(np.float32)

# Reorder the dimensions from [H, W, C] to [C, H, W]
image_tensor = np.transpose(image_norm, (2, 0, 1))

# Apply transformations and add batch dimension
image_tensor = np.expand_dims(image_tensor, axis=0) # Shape: [1, C, H, W]

- Load trained Faster R-CNN model using ONNX Runtime

In [ ]:
# Load the ONNX model
MODEL_NAME = "model/fasterrcnn_resnet50_fpn_v2_scissors.onnx"
onnx_model = onnx.load(MODEL_NAME)
onnx.checker.check_model(onnx_model)

# Initialize inference with ONNX Runtime
ort_session = ort.InferenceSession(MODEL_NAME)

- run inference on input tensor to ONNX Faster R-CNN model

In [ ]:
# run inference
outputs = ort_session.run(None, {"input": image_tensor})
print("ONNX model outputs:", outputs)

- show detection result

In [ ]:
import matplotlib.pyplot as plt
import utils

utility = utils.Utils()

CLASS_LABELS = ["background", "scissors"]  # Modify according to your dataset
THRESHOLD = 0.5 # Only display boxes above the confidence threshold


image_with_box = utility.postprocess_onnx_frcnn(
                        outputs, # ONNX prediction result 
                        original_image.copy(), # Original image
                        CLASS_LABELS, 
                        confThreshold = THRESHOLD, 
                        font_size=0.8, 
                        color=(255,127,0), 
                        text_color=(255,255,255), 
                        input_size=[224,224] # Input tensor size
                        ) 

# show result 
cv2.imshow("detection result", image_with_box)
cv2.waitKey()
cv2.destroyAllWindows()